<a href="https://colab.research.google.com/github/piyush-98/NSFW_content_moderation/blob/master/nsfw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls

drive  sample_data


In [1]:
!cp "/content/drive/My Drive/test.zip" "/content/"

In [2]:
!cp "/content/drive/My Drive/model2.h5" "/content/"

In [3]:
!unzip "/content/test.zip"

Archive:  /content/test.zip
  inflating: nsfw/0018f3e0-9446-4567-9917-ff5b46ffa20b.jpg  
  inflating: nsfw/005c70bc-b0e4-428f-b531-851adb5a4ecd.jpg  
  inflating: nsfw/005f5a5a-42d2-4476-a5bc-1fe22b6dddfd.jpg  
  inflating: nsfw/005f8b99-4ee4-4980-8847-e0ce84b306f0.jpg  
  inflating: nsfw/006d6dc0-afd1-48b7-b8b7-73dc640e725f.jpg  
  inflating: nsfw/00a68f35-f373-4f73-b862-6f3fb10f6bb4.jpg  
  inflating: nsfw/00bbd690-c237-4ef8-9349-524d2b7501d6.jpg  
  inflating: nsfw/00c4efb3-7114-457d-b2b1-46054015b2d2.jpg  
  inflating: nsfw/00f26452-fde8-48ac-a2cd-4b240f6d5b8b.jpg  
  inflating: nsfw/010a5223-ef1b-448f-9387-04d0b68647dc.jpg  
  inflating: nsfw/01280a46-bc0b-47ae-ae44-834fcda981d8.jpg  
  inflating: nsfw/01352ca3-31ea-4a56-a929-e88973714a0c.jpg  
  inflating: nsfw/01850492-48fc-4098-83de-18e61d713871.jpg  
  inflating: nsfw/02576005-443e-4764-bc0f-180df6086cd7.jpg  
  inflating: nsfw/02c4548e-98ca-4526-bdca-b746b8b7b33f.jpg  
  inflating: nsfw/02d1a84a-460b-42bc-b4bb-0f9a12a3b6fe.jp

In [ ]:
!ls

drive  model2.h5  nsfw	sample_data  sfw  test.zip


In [ ]:
# from PIL import Image
# import os
# import numpy as np
# files=os.listdir("images")
# data=[]
# labels=[]
# from PIL import ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True
# for folder in files:
#   path='images/'+folder+"/"
#   for file_ in os.listdir(path):
#       path_ = path+file_
#       im=Image.open(path_)
#       im = im.resize((100,100))
#       im=np.asarray(im)
#       im=im/255.0
#       data.append(im)
#       labels.append(folder)

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


In [ ]:
len(data)

21384

In [ ]:
import pandas as pd
df=pd.DataFrame({'x':data,'y':labels})

In [ ]:
del labels

In [ ]:
del data

In [ ]:
df1=df.sample(frac = 1) 

In [ ]:
del df

In [ ]:
df1.shape

(21384, 2)

In [ ]:
y=np.array(df1['y'])

In [ ]:
del df1['y']

In [ ]:
df1.head()

,x
16990,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
304,"[[[0.6980392156862745, 0.8235294117647058, 0.9..."
6432,"[[[0.9137254901960784, 0.8470588235294118, 0.8..."
20372,"[[[0.5529411764705883, 0.39215686274509803, 0...."
10542,"[[[0.24313725490196078, 0.35294117647058826, 0..."


In [ ]:
x=np.array(df1['x'])

In [ ]:
df1['x'].iloc[0].shape

(100, 100)

In [ ]:
!ls

drive  images  imagesClean.zip	sample_data


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
import tensorflow as tf
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

model2 = Sequential()
model2.add(Input(shape=(100,100,3)))
model2.add(Conv2D(32, (5, 5),activation='relu',padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(32, (3, 3),activation='relu',padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(64, (3, 3),activation='relu',padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(64, (3, 3),activation='relu',padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))   
model2.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))   
model2.add(Flatten())
model2.add(Dense(512,activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(128,activation='relu'))          
model2.add(Dense(5,activation='softmax'))
model2.summary()


train_path='images/'
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                            validation_split=0.2)
       
train_data_gen = train_gen.flow_from_directory(train_path,
                                               target_size=(100,100),
                                               class_mode='categorical',
                                               batch_size=32,
                                               subset="training")

val_data_gen =  train_gen.flow_from_directory(train_path,
                                              target_size=(100,100),
                                              class_mode='categorical',
                                              batch_size=32,
                                              subset="validation")

model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model2.fit_generator(train_data_gen,
                                 steps_per_epoch=200,
                                 epochs=6,
                                 validation_data=val_data_gen)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 100, 100, 32)      2432      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 50, 50, 32)        9248      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 25, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 12, 12, 64)       

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


200/200 [==============================] - 367s 2s/step - loss: 1.5400 - accuracy: 0.2731 - val_loss: 1.5842 - val_accuracy: 0.2908
Epoch 2/6
200/200 [==============================] - ETA: 0s - loss: 1.4765 - accuracy: 0.3288

KeyboardInterrupt: ignored

In [ ]:
from google.colab import files
# 
uploaded=files.upload()
for fn in uploaded.keys():
    print('user uploaded file "{name}" with length {length} bytes'.format(name=fn,length=len(uploaded[fn])))

Saving model2.h5 to model2.h5
user uploaded file "model2.h5" with length 5134064 bytes


In [4]:
from tensorflow.keras.models import load_model

In [5]:
!ls

drive  model2.h5  nsfw	sample_data  sfw  test.zip


In [6]:
model = load_model('model2.h5')

In [7]:
!ls

drive  model2.h5  nsfw	sample_data  sfw  test.zip


In [29]:
import cv2
import numpy as np
arr=[]
def predict(img_list):
    for path in img_list:
        img = cv2.imread(path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(100,100))
        img = img/255
        img = img.reshape((100,100,3))
        arr.append(img)
        #pred = np.argmax(model.predict(img))
        #results.append(pred)
    #return results

In [30]:
from PIL import Image
import os

path = "sfw/"
images_path_ = os.listdir(path)

images_path_ = [path+path_ for path_ in images_path_]
data_ = {1:0,0:0}
# list_img_ = predict(images_path_)
predict(images_path_)
# for i in list_img_:
#     data_[i]+=1
    
# print(data_)


In [32]:
from PIL import Image
import os

path = "nsfw/"
images_path_ = os.listdir(path)

images_path_ = [path+path_ for path_ in images_path_]
data_ = {1:0,0:0}
# list_img_ = predict(images_path_)
predict(images_path_)
# for i in list_img_:
#     data_[i]+=1
    
# print(data_)


In [50]:
res=np.array(arr)
res.shape

(4000, 100, 100, 3)

In [41]:
pred = (model.predict(res))

In [43]:
pred_labels=pred.argmax(axis=-1)

In [53]:
actual_labels=np.zeros(4000,)

In [54]:
actual_labels[3999]

0.0

In [55]:
actual_labels[0:2000]=1

In [59]:
from sklearn.metrics import classification_report

In [61]:
print(classification_report(actual_labels,pred_labels))

              precision    recall  f1-score   support

         0.0       0.85      0.91      0.88      2000
         1.0       0.91      0.84      0.87      2000

    accuracy                           0.88      4000
   macro avg       0.88      0.88      0.88      4000
weighted avg       0.88      0.88      0.88      4000



# New Section